In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
from sklearn.preprocessing import LabelBinarizer


# Cargar imágenes y hacer PCA

In [ ]:
## Cargo los dos numpy arrays que representan los datasets de train y test, estan en la carpeta datasets del repositorio
train = np.load("datasets/train.npy")
test = np.load("datasets/test.npy")


In [ ]:
def displayData(data, *width):
    '''
    Display data in a 2-dimensional grid
    '''

    # Set ex_width
    if width:
        ex_width = width[0]
    else:
        ex_width = int(np.sqrt(np.shape(data)[1]));

    # Compute dimensions
    (n_examples, n) = np.shape(data);
    ex_height = int(n / ex_width);
    n_rows = int(np.floor(np.sqrt(n_examples)));
    n_cols = int(np.ceil(n_examples / n_rows));

    # Set padding
    pad = 1;

    # Core
    grid = np.zeros( (pad + n_rows * (ex_height + pad),
                                pad + n_cols * (ex_width + pad)) );    
    cur = 0; # current example
    for j in range(0, n_rows):
        if cur >= n_examples: break;
        for i in range(0, n_cols):
            if cur >= n_examples: break;
            max_val = np.max(np.abs(data[cur, :]))
            from_row = pad + j * (ex_height + pad); to_row = from_row + ex_height
            from_col = pad + i * (ex_width + pad); to_col = from_col + ex_width
            grid[from_row:to_row,from_col:to_col] = \
                data[cur, :].reshape( (ex_height, ex_width) ) / max_val
            cur += 1
        

    # Display data
    if(n_examples<100):
        fig = plt.figure()
    else:
        fig = plt.figure(figsize=(n_examples/10,n_examples/20))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow((-1)*grid, extent=[0, 1, 0, 1], cmap='Greys')

    plt.show()

In [ ]:
displayData(train[:201],100)


In [ ]:
displayData(test[:201],100)



In [ ]:
# funcion que devuelve el PCA de un dataset, es decir, por cada foto (fila) va a devolver <n_componentes> cantidad de columnas.
# esto es lo que se usa de entrada
def pca_faces(n_componentes, dataset_train, dataset_test):
    pca = PCA(n_components= n_componentes)
    pca.fit(dataset_train)
    U = pca.components_
    Z_train = pca.transform(dataset_train)
    Z_test = pca.transform(dataset_test)
    
    return Z_train, Z_test

In [ ]:
# hago el pca para las fotos de train

pca_train, pca_test = pca_faces(30, train, test)

print(pca_test.shape)

# Backpropagation

In [ ]:
# y_train_ contiene los nombres de la persona de cada foto EN ORDEN como aparecen en el dataset (ver mas arriba donde estan mostradas las fotos)

X_train = pca_train
X_test = pca_test
y_train_ = ['nestor', 'claudia', 'oscar', 'eduardo', 'andres', 'eduardo', 'lujan', 'claudia', 'maira', 'eduardo', 'marcelo',
           'marcelo t.','eduardo', 'marcelo', 'eduardo', 'marcelo', 'eduardo', 'maira', 'silvia', 'marisa', 'sebastian', 'marcelo t.',
           'geronimo', 'eduardo', 'oscar', 'lujan', 'jiang', 'eduardo', 'maira', 'eduardo', 'sebastian', 'nestor', 'fernanda',
           'hernan', 'geronimo', 'eduardo', 'elemir', 'julieta', 'silvia', 'eduardo', 'josefina', 'marcelo t.', 'marcelo t.', 'hernan',
           'julieta', 'josefina', 'marcelo t.', 'lujan', 'hernan', 'jiang', 'nestor', 'eduardo', 'jiang', 'marcelo t.', 'eduardo',
           'joaquin', 'hernan', 'jiang', 'julieta', 'marisa', 'eduardo', 'eduardo', 'fernanda', 'marcelo', 'jiang', 'nestor',
           'josefina', 'eduardo', 'lujan', 'rodrigo', 'rodrigo', 'sebastian', 'marcelo', 'andres', 'marcelo', 'julieta', 'hernan',
           'rodrigo', 'maribel', 'eduardo', 'fernanda', 'fernanda', 'rodrigo', 'marisa', 'maribel', 'eduardo', 'jiang', 'eduardo',
           'julieta', 'silvia', 'eduardo', 'eduardo', 'marcelo t.', 'marcelo t.', 'fernanda', 'joaquin', 'lujan', 'jiang', 'eduardo',
           'andres', 'maira', 'nestor', 'jiang', 'claudia', 'nestor', 'elemir', 'hernan', 'maira', 'hernan', 'eduardo', 
           'andres', 'marisa', 'eduardo', 'hernan', 'rodrigo', 'eduardo', 'elemir', 'josefina', 'andres', 'marcelo t.', 'josefina']

y_test_ = ['claudia', 'sebastian', 'nestor', 'andres', 'jiang', 'maribel', 'hernan', 'marcelo', 'marisa', 'rodrigo',
           'elemir', 'josefina', 'julieta', 'geronimo', 'eduardo', 'fernanda', 'lujan', 'marcelo t.', 'silvia',
           'joaquin', 'oscar']

# para poder usar esta lista de nombres, debo meterlas como un array de 0s y 1s.
# en la celda de abajo se ve mas claro pero aca esta la transformacion hecha
y_train = LabelBinarizer().fit_transform(y_train_)
y_test = LabelBinarizer().fit_transform(y_test_)

index = 12

y_test = np.insert(y_test, index, values=np.zeros(y_test.shape[0]), axis=1)



In [ ]:
# imprimo cada elemento del vector de nombres con su correspondiente array de 0 y 1.
# por ejemplo el array que tenga un 1 en la posicion 2 significa "claudia"
for i in range(10):
    print(f"{y_train_[i]}:{y_train[i]}")

In [ ]:
for i in range(21):
    print(f"{y_test_[i]}:{y_test[i]}")

In [ ]:
import pandas as pd
pd.Series(y_train_).value_counts()
pd.Series(y_test_).value_counts()
## esto es para ver cuantas fotos hay de cada persona. Si hay menos fotos va a tener menos chance de reconocer a cada persona.
# Nuestro dataset le faltan fotos, por eso hay menos fotos por persona.

y_test.shape


## Entrenamiento de la red

In [ ]:
## aca arranca todo el script, marco con ####### las partes que modifique del original


def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + np.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + math.exp(-2.0*x) ) - 1.0
    return y

func_eval_vec = np.vectorize(func_eval)

def deriv_eval(fname, y):  
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y*(1.0-y)
        case "tansig":
            d = 1.0 - y*y
    return d

deriv_eval_vec = np.vectorize(deriv_eval)

# La entrada ahora va a ser X_train que es la matriz  con el PCA de las fotos de train
###############################################
entrada =  X_train 
###############################################


# La salida ahora es y_train, un array de 0s y 1s por cada foto, indicando a quien pertenece como vimos arriba.
###############################################
salida = y_train
###############################################


# Paso las listas a numpy
X = np.array(entrada)
Y = np.array(salida)

filas_qty = len(X)
###########################
# Cantidad de neuronas de entradas: 1 por Componente principal
input_size = X.shape[1]   # 1 por CP
###########################


hidden_size = 10  # neuronas capa oculta

# Cantidad de neuronas de salida: 1 por persona en train
###########################
output_size = Y.shape[1] 
###########################

# defino las funciones de activacion de cada capa
hidden_FUNC = 'logsig'  # uso la logistica

###############################################
output_FUNC = 'logsig'  # uso la logistica
###############################################


# incializo los graficos
#grafico = perceptron_plot(X, np.array(salida), 0.0)

# Incializo las matrices de pesos azarosamente
# W1 son los pesos que van del input a la capa oculta
# W2 son los pesos que van de la capa oculta a la capa de salida
np.random.seed(1021) #mi querida random seed para que las corridas sean reproducibles
W1 = np.random.uniform(-0.5, 0.5, [hidden_size, input_size])
X01 = np.random.uniform(-0.5, 0.5, [hidden_size, 1] )
W2 = np.random.uniform(-0.5, 0.5, [output_size, hidden_size])
X02 = np.random.uniform(-0.5, 0.5, [output_size, 1] )

# Avanzo la red, forward
# para TODOS los X al mismo tiempo ! 
#  @ hace el producto de una matrix por un vector_columna
hidden_estimulos = W1 @ X.T + X01
hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
output_estimulos = W2 @ hidden_salidas + X02
output_salidas = func_eval_vec(output_FUNC, output_estimulos)

# calculo el error promedi general de TODOS los X
Error = np.mean( (Y.T - output_salidas)**2 )
print(f"Error inicial {Error}")

# Inicializo
epoch_limit = 2000    # para terminar si no converge
Error_umbral = 1.0e-06
learning_rate = 0.2
Error_last = 10    # lo debo poner algo dist a 0 la primera vez
epoch = 0

while ( math.fabs(Error_last-Error)>Error_umbral and (epoch < epoch_limit)):
    epoch += 1
    Error_last = Error
    
    # recorro siempre TODA la entrada
    for fila in range(filas_qty): #para cada input x_sub_fila del vector X
        # propagar el x hacia adelante
        hidden_estimulos = W1 @ X[fila:fila+1, :].T + X01
        hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
        output_estimulos = W2 @ hidden_salidas + X02
        output_salidas = func_eval_vec(output_FUNC, output_estimulos)
        
        # calculo los errores en la capa hidden y la capa output
        ErrorSalida = Y[fila:fila+1,:].T - output_salidas
        # output_delta es un solo numero
        output_delta = ErrorSalida * deriv_eval_vec(output_FUNC, output_salidas)
        # hidden_delta es un vector columna
        hidden_delta = deriv_eval_vec(hidden_FUNC, hidden_salidas)*(W2.T @ output_delta)

        # ya tengo los errores que comete cada capa
        # corregir matrices de pesos, voy hacia atras
        # backpropagation
        W1 = W1 + learning_rate * (hidden_delta @ X[fila:fila+1, :] )
        X01 = X01 + learning_rate * hidden_delta
        W2 = W2 + learning_rate * (output_delta @ hidden_salidas.T)
        X02 = X02 + learning_rate * output_delta

    # ya recalcule las matrices de pesos
    # ahora avanzo la red, feed-forward
    hidden_estimulos = W1 @ X.T + X01
    hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
    output_estimulos = W2 @ hidden_salidas + X02
    output_salidas = func_eval_vec(output_FUNC, output_estimulos)
    print(hidden_estimulos.shape)
    # calculo el error promedio general de TODOS los X
    Error= np.mean( (Y.T - output_salidas)**2 )
    
    # Imprimo el error en cada epoch
    ###############################################
    print(f"epoch: {epoch} - error: {Error}")
    ###############################################

## Predicciones

## Train

In [ ]:

prediccion_train = []
personas = np.unique(y_train_)

for fila in range(len(y_train_)):

    hidden_estimulos_predict = W1 @ X[fila:fila+1, :].T + X01
    hidden_salidas_predict = func_eval_vec(hidden_FUNC, hidden_estimulos_predict)
    output_estimulos_predict = W2 @ hidden_salidas_predict + X02
    output_salidas_predict = func_eval_vec(output_FUNC, output_estimulos_predict)
    prediccion_train.append(np.argmax(output_salidas_predict))

prediccion_train = np.array(prediccion_train)
y_train_ = np.array(y_train_)
aciertos_train = []
cant_aciertos_train = 0
for indice in range(len(y_train_)):
    aciertos_train.append(personas[prediccion_train[indice]])

for x, y in zip(aciertos_train, y_train_):
    if x == y:
        cant_aciertos_train += 1
        



In [ ]:

print("Cantidad de aciertos: ", cant_aciertos_train)
print("Accuracy del training:", round(cant_aciertos_train/len(aciertos_train),3))
print("Real - Predicción")
for indice in range(len(aciertos_train)):
    print(y_train_[indice], " - ", aciertos_train[indice])


## Test

In [ ]:
X_t = np.array(X_test)
prediccion_test = []

for fila in range(len(y_test_)):

    hidden_estimulos_predict = W1 @ X_t[fila:fila+1, :].T + X01
    hidden_salidas_predict = func_eval_vec(hidden_FUNC, hidden_estimulos_predict)
    output_estimulos_predict = W2 @ hidden_salidas_predict + X02
    output_salidas_predict = func_eval_vec(output_FUNC, output_estimulos_predict)
    prediccion_test.append(np.argmax(output_salidas_predict))

prediccion_test = np.array(prediccion_test)
y_test_ = np.array(y_test_)
aciertos_test = []
cant_aciertos_test = 0
for indice in range(len(y_test_)):
    aciertos_test.append(personas[prediccion_test[indice]])

for x, y in zip(aciertos_test, y_test_):
    if x == y:
        cant_aciertos_test += 1
        
print("Cantidad de aciertos: ", cant_aciertos_test)
print("Accuracy del training:", round(cant_aciertos_test/len(aciertos_test),3))
print("Real - Predicción")
for indice in range(len(aciertos_test)):
    print(y_test_[indice], " - ", aciertos_test[indice])


# Tensorflow

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



## Creación de la red

In [ ]:
escala = np.max([X_train.max(), X_test.max()])

train_tensor = X_train / escala
test_tensor = X_test / escala

cant_personas = len(np.unique(y_train_))

# capa oculta con 10 neuronas
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(30, 1)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(cant_personas)
])

model.compile(optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=learning_rate),
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])



## Entrenamiento de la red

In [ ]:
model.fit(train_tensor, y_train, epochs=epoch_limit)

In [ ]:
test_loss, test_acc = model.evaluate(test_tensor,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)


## Predicciones Tensorflow

In [ ]:
# Se agrega "Maira" debido a que no hay una foto suya en los datos de test.
predictions = model.predict(test_tensor)


In [ ]:
aciertos = []
cant_aciertos = 0
for indice in range(len(y_test_)):
    aciertos.append(personas[predictions[indice,:].argmax()])

for x, y in zip(aciertos, y_test_):
    if x == y:
        cant_aciertos += 1
        
    

print("Cantidad de aciertos: ",cant_aciertos)
print("Cantidad fotos de testing: ",len(y_test_))
print("Accuracy: ", round(cant_aciertos/len(y_test_), 3))

In [ ]:
print("Real - Predicción")
for indice in range(len(aciertos)):
    print(y_test_[indice], " - ", aciertos[indice])